In [4]:
import pathlib 
import numpy as np 

In [5]:
T_0 = 253  # initial temperature 

temperatures = np.arange(220,  235, 5 ) 
temperatures

array([220, 225, 230])

In [6]:
240-273.15

-33.14999999999998

# Templates 

### nvt.mdp 

In [7]:
def render_nvt_mdp(temperature: int): 
    file = f'''title       = Umbrella equilibration
define      = -DFLEXIBLE
; Run parameters
integrator  = md
dt          = 0.002
tinit       = 0
nsteps      = 1000000     ; 2000 ps
nstcomm     = 10
; Output parameters
nstvout             = 25000  ; every 25 ps
nstfout             = 25000
nstxout-compressed  = 25000
nstenergy           = 2500
; Bond parameters
; Single-range cutoff scheme
cutoff-scheme   = Verlet
nstlist         = 100
ns_type         = grid
rlist           = 1.2
rcoulomb        = 1.2
rvdw            = 1.2
; PME electrostatics parameters
coulombtype     = PME
fourierspacing  = 0.12
fourier_nx      = 0
fourier_ny      = 0
fourier_nz      = 0
pme_order       = 4
ewald_rtol      = 1e-5
optimize_fft    = yes
; Langevin dynamics is a thermostat
; tau -t is the inverse friction coefficient
tcoupl	    = Nose-Hoover
tc_grps     = Water non-Water
tau_t       = 0.4     10
ref_t       = {temperature}     {temperature}
; Pressure coupling is on
Pcoupl          = no
refcoord_scaling = com
; Generate velocities is off
; Periodic boundary conditions are on in all directions
pbc     = xyz
; Long-range dispersion correction
DispCorr    = EnerPres
gen_vel                  = yes
gen_temp                 = 253
gen_seed                 = -1
;pcoupl                   = No ; berendsen
;pcoupltype               = anisotropic
;tau-p                    = 4.0
;compressibility          = 5.0e-5 5.0e-5 5.0e-5 0.0 0.0 0.0
;ref-p                    = 0.0 0.0 0.0 0.0 0.0 0.0
constraints              = hbonds
constraint-algorithm     = lincs
lincs-order              = 4
lincs-iter               = 1
continuation             = no
freezegrps               = non-Water
freezedim                = Y Y Y

'''
    return file 

### run.sh 

In [8]:
# def render_run_sh(temperatures): 
#     file = f'''#!/bin/bash
# #SBATCH --job-name=slab1
# #SBATCH --account=project_2008684
# #SBATCH --time=01:00:00
# #SBATCH --partition=small
# #SBATCH --ntasks=1
# #SBATCH --cpus-per-task=8

# set -o errexit

# module load gromacs-env

# '''
#     for temperature in temperatures: 
#         executions = f'''
# gmx_mpi grompp -f nvt_{temperature}.mdp -c minimized.gro -p topol.top -n index2.ndx -o {temperature}nvt.tpr
# srun gmx_mpi mdrun -v -deffnm {temperature}nvt
#     '''
#         file = file+executions
    
#     return file 

In [9]:
def render_run_sh_sbatch(temperature): 
    file = '''#!/bin/bash
#SBATCH --job-name=slab1
#SBATCH --account=project_2008684
#SBATCH --time=20:00:00
#SBATCH --partition=small
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=8

set -o errexit

module load gromacs-env

export OMP_NUM_THREADS=${SLURM_CPUS_PER_TASK}


'''

    executions = f'''
gmx_mpi grompp -f nvt_{temperature}.mdp -c minimized.gro -p topol.top -n index2.ndx -o {temperature}nvt.tpr
srun gmx_mpi mdrun -v -deffnm {temperature}nvt -ntomp $OMP_NUM_THREADS
'''
    
    
    file = file+executions
    
    return file 

In [10]:
print(render_run_sh_sbatch(temperature=200))

#!/bin/bash
#SBATCH --job-name=slab1
#SBATCH --account=project_2008684
#SBATCH --time=20:00:00
#SBATCH --partition=small
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=8

set -o errexit

module load gromacs-env

export OMP_NUM_THREADS=${SLURM_CPUS_PER_TASK}



gmx_mpi grompp -f nvt_200.mdp -c minimized.gro -p topol.top -n index2.ndx -o 200nvt.tpr
srun gmx_mpi mdrun -v -deffnm 200nvt -ntomp $OMP_NUM_THREADS



# Execution 

In [11]:
# remove pre-existing .mdp files 
old_mdp_paths = list(pathlib.Path(".").glob("nvt_*.mdp"))
for old_mdp_path in old_mdp_paths: 
    old_mdp_path.unlink()



# remove pre-existing .sh files 
old_paths = list(pathlib.Path(".").glob("run_*.sh"))
for old_path in old_paths: 
    old_path.unlink()


In [12]:
# create input files with different temperatures 
for temperature in temperatures: 
    input = f"nvt_{temperature}.mdp"
    with open(input, "w") as file: 
        file.write(render_nvt_mdp(temperature))



In [13]:
# create run_{temperature}.sh files

for temperature in temperatures: 
    with open(f"run_{temperature}.sh", 'w') as file: 
        file.write(render_run_sh_sbatch(temperature=temperature))
